In [154]:
import requests
from bs4 import BeautifulSoup
import json

def read_home():
    url = f'https://iniciativaliberal.pt/legislativas-2022/programa-eleitoral/'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    proposals = soup.find_all('a')

    output = []
    for proposal in proposals:
        try:
            href = proposal.attrs['href']
            if not href.startswith('https://iniciativaliberal.pt/programa2022/'):
                continue
        except KeyError:
            continue
        
        response = requests.get(href)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        section = soup.find('h3', class_='brxe-heading')
        name = soup.find('h1').text
    
        container = soup.find('div', class_='brxe-post-content')

        tag = container.h2
        items = {}
        label, body = None, None
        while True:   
            if not tag:
                items[label] = body
                break
            
            if tag.name == 'h2':
                items[label] = body
                label = tag.text
                body = None
                if tag.text in ('QUESTÕES FREQUENTES', 'PERGUNTAS FREQUENTES'):
                    break
            elif tag.name == 'p':
                body = tag.text
            elif tag.name == 'ol':
                array = [x.text for x in tag.find_all('li')]
                body = array
            elif tag.name == 'ul':
                array = [x.text for x in tag.find_all('li')]
                body = array
                
            tag = tag.next_sibling
            

        items = {k: v for k, v in items.items() if k}
        
        output.append({
            'name': name,
            'chapter': section.text.strip(),
            'content': items.get('PROPOSTA'),
            'objective': items.get('OBJETIVOS'),
            'rationale': items.get('RACIONAL'),
        })
                
    return output

In [155]:
data = read_home()

[{'name': 'Qualidade e Transparência da Legislação',
  'chapter': 'Justiça',
  'content': ['Determinar, de forma inequívoca, o dever de fundamentação expressa e completa dos atos legislativos, incluindo do ponto de vista dos problemas que se pretende resolver, dos benefícios esperados e dos custos e riscos inerentes, bem com dos mitigantes dos referidos riscos.',
   'Incrementar a elaboração sistemática de versões atualizadas dos diplomas sujeitos a revisões frequentes e subsequente disponibilização online de forma centralizada, no site da internet do Diário da República.',
   'Fixação de um regime único de legística, que substitua os atualmente vigentes, e harmonize, formalmente, os atos legislativos, conferindo-lhes, estruturas semelhantes, determinando os conjuntos de referências obrigatórias, definindo os cuidados a ter na elaboração e as exigências a ter em conta nas versões finais.',
   'Assegurar a disponibilidade efetiva de apoio técnico especializado para revisãode legística d

In [156]:
with open('il-2022.json', 'w') as f:
    f.write(json.dumps(data, ensure_ascii=False))